In [23]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import TruncatedSVD
import stanza
from sklearn.decomposition import PCA

# Part 1: Load data

In [24]:
df = pd.read_csv('mix_combined.csv')
df.head()

,user_id,languages,num of languages,summary,education,num of education,experiences,num of experiences,skill_k50_accounting / financial reporting / auditing,skill_k50_analysis / financial analysis / finance,...,language_taiwanese,language_tamil,language_telugu,language_thai,language_tigrinya,language_turkish,language_ukrainian,language_urdu,language_vietnamese,combined_text
0,1033137,"['english', 'spanish']",2.0,Director of biopharma equity research and publ...,NYU Stern School of Business\nMaster of Busine...,3.0,Evercore : Director - Biotechnology & Pharmace...,6.0,1,1,...,0,0,0,0,0,0,0,0,0,summary: Director of biopharma equity research...
1,1098586,[],0.0,blank,Fordham Gabelli School of Business\nMaster of ...,1.0,- : Research Analyst\nNone.-.\nNo description\...,3.0,0,1,...,0,0,0,0,0,0,0,0,0,summary: blank;education: Fordham Gabelli Scho...
2,1115736,[],0.0,I am the CEO of JLab. We are the fastest growi...,University of Oklahoma\nNA : NA\n1995-1999\n\n...,2.0,JLab Audio : President\n2011.11-.\nNo descript...,7.0,0,0,...,0,0,0,0,0,0,0,0,0,summary: I am the CEO of JLab. We are the fast...
3,1341457,[],0.0,I offer ~20 years’ experience on the evolution...,University of Wisconsin-Madison\nBBA : Finance...,3.0,SLR Capital Partners : Partner\n2023.1-.\nNo d...,6.0,0,1,...,0,0,0,0,0,0,0,0,0,summary: I offer ~20 years’ experience on the ...
4,1501168,[],0.0,We specialize in providing long-term financial...,University of Arkansas\nBachelor of Science (B...,1.0,"Boston Mountain Money Management, Inc. : Princ...",4.0,1,1,...,0,0,0,0,0,0,0,0,0,summary: We specialize in providing long-term ...


# Part 2: Preprocess combined_text

In [25]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stanza.download('en', processors='lemma')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tina\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


2024-04-28 14:34:46 INFO: Downloaded file to C:\Users\Tina\stanza_resources\resources.json
2024-04-28 14:34:46 INFO: Downloading these customized packages for language: en (English)...
| Processor | Package           |
---------------------------------
| lemma     | combined_nocharlm |

2024-04-28 14:34:47 INFO: File exists: C:\Users\Tina\stanza_resources\en\lemma\combined_nocharlm.pt
2024-04-28 14:34:47 INFO: Finished downloading models and saved to C:\Users\Tina\stanza_resources


In [26]:
# Didn't do stemming since it would cost overstemming problem.
#lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()    
    # Remove URLs, mentions, and hashtags
    text = re.sub(r'http\S+|@\S+|#\S+', '', text)
    # Remove punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Perform lemmatization
    doc = nlp(text)
    lemmatized_tokens = [word.lemma for sent in doc.sentences for word in sent.words]
    #lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    # Join the preprocessed tokens back into a string
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text

nlp = stanza.Pipeline('en', processors='tokenize,lemma')

2024-04-28 14:34:47 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-04-28 14:34:49 INFO: Downloaded file to C:\Users\Tina\stanza_resources\resources.json
2024-04-28 14:34:49 WARNING: Language en package default expects mwt, which has been added
2024-04-28 14:34:49 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| lemma     | combined_nocharlm |

2024-04-28 14:34:49 INFO: Using device: cpu
2024-04-28 14:34:49 INFO: Loading: tokenize
2024-04-28 14:34:49 INFO: Loading: mwt
2024-04-28 14:34:49 INFO: Loading: lemma
2024-04-28 14:34:49 INFO: Done loading processors!


In [27]:
df['preprocessed_combined_text']=df['combined_text'].apply(preprocess_text)

In [28]:
df.loc[0, 'preprocessed_combined_text']
#df.loc[0,'combined_text']

'summary director of biopharma equity research and publishing analyst on the rank sellside team on wall street biotechpharma industry veteran with prove success at analyze clinical trial assess drug market potential and at cultivate relationship with physicianthought leader in order to drive strategic business objective extremely knowledgeable about the fda drug approval process and other key driver of the biopharma industry education nyu stern school of business master of business administration mba finance management fev fundao getulio vargas to do business in brazil rutger university pharmd pharmacy nonenoneexperiencence evercore director biotechnology pharmaceuticalsmajor equity research no description evercore vice president biotechnology pharmaceuticalsmajor equity research cover biotech major pharma and specialty pharma company under head analyst umer raffat and previously under dr mark schoenebaum ii rank in large cap biotech in large cap pharma in specialty pharma ii rank in l

## Bag-of-word

In [29]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
bow_matrix = vectorizer.fit_transform(df['preprocessed_combined_text'])
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

whole_bow_df = pd.concat([bow_df,df],axis=1)

print('The number of columns in df is:', df.shape[1], \
      'The number of columns in bow_df is:', bow_df.shape[1],\
      'The number of columns in whole_bow_df is:', whole_bow_df.shape[1])

MemoryError: Unable to allocate 8.58 GiB for an array with shape (3388, 339885) and data type int64

## TF-IDF

In [ ]:
tfidf_transformer = TfidfTransformer()
tfidf_matrix = tfidf_transformer.fit_transform(df['preprocessed_combined_text'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

whole_tfidf_df = pd.concat([tfidf_df,df],axis=1)
print('The number of columns in df is:', df.shape[1], \
      'The number of columns in tfidf_df is:', tfidf_df.shape[1],\
      'The number of columns in whole_tfidf_df is:', whole_tfidf_df.shape[1])

# Part 3: Reduce dimension

In [ ]:
selected_columns = whole_bow_df.columns[whole_bow_df.columns.str.startswith('skill_k50')]
bow_df_x = whole_bow_df.drop(selected_columns, axis=1)
# drop string columns in whole_bow_df
bow_df_x=bow_df_x.drop(['combined_text','preprocessed_combined_text','languages','summary','education','experiences','user_id'],axis=1)
bow_df_y = whole_bow_df.loc[:, selected_columns]

50

In [ ]:
selected_columns = whole_tfidf_df.columns[whole_bow_df.columns.str.startswith('skill_k50')]
tfidf_df_x = whole_tfidf_df.drop(selected_columns, axis=1)
# drop string columns in whole_bow_df
tfidf_df_x=tfidf_df_x.drop(['combined_text','preprocessed_combined_text','languages','summary','education','experiences','user_id'],axis=1)
tfidf_df_y = whole_tfidf_df.loc[:, selected_columns]

## Truncated SVD

In [ ]:
trunc = TruncatedSVD(n_components=200)

bow_array_x = bow_df_x.values
# Fit and transform the bow_matrix
bow_array_trunc = trunc.fit_transform(bow_array_x)

new_column_names = [str(i + 1) for i in range(trunc.n_components)]
# Convert the transformed matrix to a DataFrame
bow_df_trunc_x = pd.DataFrame(bow_array_trunc, columns=new_column_names)
bow_df_trunc_x.head()

AttributeError: 'list' object has no attribute 'keys'

In [ ]:
# Fit and transform the bow_matrix
tfidf_array_x = tfidf_df_x.values

tfidf_matrix_trunc = trunc.fit_transform(tfidf_array_x)

new_column_names = [str(i + 1) for i in range(trunc.n_components)]

# Convert the transformed matrix to a DataFrame
tfidf_df_trunc_x = pd.DataFrame(tfidf_matrix_trunc, columns=new_column_names)

## PCA

In [ ]:
# Create an instance of PCA
pca = PCA(n_components=200)

bow_array_x = bow_df_x.values
# Fit and transform the bow_matrix
bow_array_pca = pca.fit_transform(bow_array_x)

# Create a list of column names as 1, 2, 3, ..., n_components
new_column_names = [str(i + 1) for i in range(pca.n_components)]

# Convert the transformed matrix to a DataFrame
bow_df_pca_x = pd.DataFrame(bow_array_pca, columns=new_column_names)

In [ ]:
# Create an instance of PCA

tfidf_array_x = tfidf_df_x.values
# Fit and transform the bow_matrix
tfidf_array_pca = pca.fit_transform(tfidf_array_x)

# Create a list of column names as 1, 2, 3, ..., n_components
new_column_names = [str(i + 1) for i in range(pca.n_components)]

# Convert the transformed matrix to a DataFrame
tfidf_df_pca_x = pd.DataFrame(tfidf_array_pca, columns=new_column_names)

# Part 4: Machine learning models

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, accuracy_score
import lightgbm as lgb

## 1. Random Forest+bow+truncated svd

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bow_df_trunc_x, bow_df_y, test_size=0.2, random_state=42)

model_RF = RandomForestClassifier(n_estimators=100, random_state=42)
multi_target_RF = MultiOutputClassifier(model_RF, n_jobs=-1)

multi_target_RF.fit(X_train, y_train)

y_pred=multi_target_RF.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Calculate precision, recall, and F1-score
print(classification_report(y_test, y_pred, target_names=y_test.columns))

Accuracy: 0.17
              precision    recall  f1-score   support

           0       0.33      0.01      0.02        81
           1       0.90      1.00      0.95       613
           2       0.00      0.00      0.00         8
           3       0.50      0.01      0.02       115
           4       0.00      0.00      0.00        39
           5       0.50      0.02      0.04        97
           6       0.00      0.00      0.00        31
           7       0.00      0.00      0.00        34
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00       110
          10       0.00      0.00      0.00        41
          11       0.00      0.00      0.00        30
          12       0.00      0.00      0.00        22
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00        27
          16       0.00      0.00      0.00        15
          17

c:\Users\Tina\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 2. Random Forest+bow+PCA

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bow_df_pca_x, bow_df_y, test_size=0.2, random_state=42)

model_RF = RandomForestClassifier(n_estimators=100, random_state=42)
multi_target_RF = MultiOutputClassifier(model_RF, n_jobs=-1)

multi_target_RF.fit(X_train, y_train)

y_pred=multi_target_RF.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Calculate precision, recall, and F1-score
print(classification_report(y_test, y_pred, target_names=y_test.columns))

## 3. Random Forest+TF-IDF+truncated svd

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_df_trunc_x, tfidf_df_y, test_size=0.2, random_state=42)

model_RF = RandomForestClassifier(n_estimators=100, random_state=42)
multi_target_RF = MultiOutputClassifier(model_RF, n_jobs=-1)

multi_target_RF.fit(X_train, y_train)

y_pred=multi_target_RF.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Calculate precision, recall, and F1-score
print(classification_report(y_test, y_pred, target_names=y_test.columns))

## 4. Random Forest+TF-IDF+PCA

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_df_pca_x, tfidf_df_y, test_size=0.2, random_state=42)

model_RF = RandomForestClassifier(n_estimators=100, random_state=42)
multi_target_RF = MultiOutputClassifier(model_RF, n_jobs=-1)

multi_target_RF.fit(X_train, y_train)

y_pred=multi_target_RF.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Calculate precision, recall, and F1-score
print(classification_report(y_test, y_pred, target_names=y_test.columns))

## 5. LightGBM+bow+truncated svd

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bow_df_trunc_x, bow_df_y, test_size=0.2, random_state=42)

# Initialize LightGBM classifier and MultiOutputClassifier
model_LGBM = lgb.LGBMClassifier(n_estimators=100, random_state=42, learning_rate=0.1)
multi_target_LGBM = MultiOutputClassifier(model_LGBM, n_jobs=-1)

# Fit model on training data
multi_target_LGBM.fit(X_train, y_train)

# Predict on test data
y_pred = multi_target_LGBM.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred, target_names=y_test.columns))

## 6. LightGBM+bow+PCA

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(bow_df_pca_x, bow_df_y, test_size=0.2, random_state=42)

# Initialize LightGBM classifier and MultiOutputClassifier
model_LGBM = lgb.LGBMClassifier(n_estimators=100, random_state=42, learning_rate=0.1)
multi_target_LGBM = MultiOutputClassifier(model_LGBM, n_jobs=-1)

# Fit model on training data
multi_target_LGBM.fit(X_train, y_train)

# Predict on test data
y_pred = multi_target_LGBM.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred, target_names=y_test.columns))

## 7. LightGBM+TF-IDF+truncated svd

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_df_trunc_x, tfidf_df_y, test_size=0.2, random_state=42)

# Initialize LightGBM classifier and MultiOutputClassifier
model_LGBM = lgb.LGBMClassifier(n_estimators=100, random_state=42, learning_rate=0.1)
multi_target_LGBM = MultiOutputClassifier(model_LGBM, n_jobs=-1)

# Fit model on training data
multi_target_LGBM.fit(X_train, y_train)

# Predict on test data
y_pred = multi_target_LGBM.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred, target_names=y_test.columns))

## 8. LightGBM+TF-IDF+PCA

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_df_pca_x, tfidf_df_y, test_size=0.2, random_state=42)

# Initialize LightGBM classifier and MultiOutputClassifier
model_LGBM = lgb.LGBMClassifier(n_estimators=100, random_state=42, learning_rate=0.1)
multi_target_LGBM = MultiOutputClassifier(model_LGBM, n_jobs=-1)

# Fit model on training data
multi_target_LGBM.fit(X_train, y_train)

# Predict on test data
y_pred = multi_target_LGBM.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred, target_names=y_test.columns))